In [23]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import mlflow
import pickle

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
X_train.shape,X_val.shape

((73908, 13221), (61921, 13221))

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479513631630414

In [10]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [11]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.167275941179728

In [20]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Arunabha")

    mlflow.log_param("train-data-path","../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path","../data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha",alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)

    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

In [27]:
mlflow.xgboost.autolog(disable=True)
mlflow.end_run()

In [28]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    params = {
        'max_depth': 14,
        'learning_rate': 0.29015003133818035,
        'reg_alpha': 0.06436125613731614,
        'reg_lambda': 0.10286826321678876,
        'min_child_weight': 1.4608108032326945,
        'objective': 'reg:linear',
        'seed': 42
    }

mlflow.log_params(params)


booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse",rmse)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)
mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[16:05:43] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.09900
[1]	validation-rmse:12.66085
[2]	validation-rmse:10.39945
[3]	validation-rmse:8.96012
[4]	validation-rmse:8.06413
[5]	validation-rmse:7.51956
[6]	validation-rmse:7.18019
[7]	validation-rmse:6.97115
[8]	validation-rmse:6.83932
[9]	validation-rmse:6.75531
[10]	validation-rmse:6.70074
[11]	validation-rmse:6.66186
[12]	validation-rmse:6.63505
[13]	validation-rmse:6.61763
[14]	validation-rmse:6.60827
[15]	validation-rmse:6.60070
[16]	validation-rmse:6.59356
[17]	validation-rmse:6.58888
[18]	validation-rmse:6.58481
[19]	validation-rmse:6.58044
[20]	validation-rmse:6.57696
[21]	validation-rmse:6.57392
[22]	validation-rmse:6.57077
[23]	validation-rmse:6.56831
[24]	validation-rmse:6.56583
[25]	validation-rmse:6.56407
[26]	validation-rmse:6.56260
[27]	validation-rmse:6.56106
[28]	validation-rmse:6.55888
[29]	validation-rmse:6.55727
[30]	validation-rm

ModelInfo(artifact_path='models_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.9.12', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/53729d50f74e430089dc8b6076dd6c74/models_mlflow', model_uuid='9a66393d940844478b584f8e8ab1549b', run_id='53729d50f74e430089dc8b6076dd6c74', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-05-30 10:38:47.472873', mlflow_version='1.26.0')

In [29]:
mlflow.end_run()

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [16]:
mlflow.xgboost.autolog()

params = {
    'max_depth': 14,
    'learning_rate': 0.29015003133818035,
    'reg_alpha': 0.06436125613731614,
    'reg_lambda': 0.10286826321678876,
    'min_child_weight': 1.4608108032326945,
    'objective': 'reg:linear',
    'seed': 42
}

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2022/05/30 07:53:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '75236ef7b4274d899eb0e9188ec8f2ac', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[07:53:24] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.09900
[1]	validation-rmse:12.66085
[2]	validation-rmse:10.39945
[3]	validation-rmse:8.96012
[4]	validation-rmse:8.06413
[5]	validation-rmse:7.51956
[6]	validation-rmse:7.18019
[7]	validation-rmse:6.97115
[8]	validation-rmse:6.83932
[9]	validation-rmse:6.75531
[10]	validation-rmse:6.70074
[11]	validation-rmse:6.66186
[12]	validation-rmse:6.63505
[13]	validation-rmse:6.61763
[14]	validation-rmse:6.60827
[15]	validation-rmse:6.60070
[16]	validation-rmse:6.59356
[17]	validation-rmse:6.58888
[18]	validation-rmse:6.58481
[19]	validation-rmse:6.58044
[20]	validation-rmse:6.57696
[21]	validation-rmse:6.57392
[22]	validation-rmse:6.57077
[23]	validation-rmse:6.56831
[24]	validation-rmse:6.56583
[25]	validation-rmse:6.56407
[26]	validation-rmse:6.56260
[27]	validation-rmse:6.56106
[28]	validation-rmse:6.55888
[29]	validation-rmse:6.55727
[30]	validation-rm

2022/05/30 07:55:46 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/05/30 07:55:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/arunabha/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [30]:
logged_model = 'runs:/53729d50f74e430089dc8b6076dd6c74/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[16:22:00] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 53729d50f74e430089dc8b6076dd6c74

In [32]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[16:23:44] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [33]:
xgboost_model

In [37]:
y_pred = xgboost_model.predict(valid)
y_pred[:10]

array([15.002704 ,  7.0362053, 13.509653 , 24.403625 ,  9.16815  ,
       17.186832 , 11.160499 ,  8.401367 ,  8.95488  , 19.736383 ],
      dtype=float32)